In [1]:
import time
from dotenv import load_dotenv
load_dotenv()
import openai
import os
from enum import Enum

OPENAI_CHAT_MODEL = "gpt-3.5-turbo"
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()
os.environ["OPENAI_API_KEY"] = open_file('Key/openai_api_key.txt')
openai.api_key = open_file('Key/openai_api_key.txt')
openai_api_key = openai.api_key
class Color(Enum):
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    RESET = '\033[0m'

def colorize(text: str, color: Color) -> str:
    return f"{color.value}{text}{Color.RESET.value}"

SYSTEM_INSTRUCTION_TEMPLATE = "You are an expert and world-famous scientist and are my real-time assistant in a debate I'm having with someone."
USER_INSTRUCTION_TEMPLATE = "My name is $NAME and I am a $ROLE. My opponent is $PARTNER. Our shared is to $GOAL. My private goal is $PRIVATE_GOAL. When you think I have accomplished the shared goal and my private goal, say 'I think we have accomplished our goal'."

class Agent:
    def __init__(self, name: str, role: str, goal: str, private_goal: str, beliefs: str, color: Color):
        self.name = name
        self.role = role
        self.goal = goal
        self.private_goal = private_goal
        self.beliefs = beliefs
        self.partner_message = None
        self.color = color
        self.memory = None
        self.partner = None
        self.task_list = []

    def __str__(self):
        return self.name

    def set_partner(self, partner):
        self.partner = partner

    def initialize(self):
        system_instruction = SYSTEM_INSTRUCTION_TEMPLATE
        user_instruction = USER_INSTRUCTION_TEMPLATE.replace("$NAME", self.name).replace("$ROLE", self.role).replace(
            "$GOAL", self.goal).replace("$PRIVATE_GOAL", self.private_goal).replace("$PARTNER", self.partner.name)
        self.memory = [{"role": "system", "content": system_instruction}, {"role": "user", "content": user_instruction}, {"role": "user", "content": f"Here are my strongly held views: {self.beliefs}"}]

    def __repr__(self):
        return f"{self.name} ({self.description})"
    
    def create_task_list(self, query):
        instructions = f"Create a task list to solve the following query: '{query}'."
        self.memory.append({"role": "user", "content": instructions})

    def choose_task(self):
        if not self.task_list:
            return f"I need a list of tasks before I can choose one."

        selected_task = self.task_list[0]  # Choose the first task in the list for simplicity
        return f"I suggest starting with: {selected_task}"

goal = "How can we align an AI?"
agent1 = Agent("GPT-6", "Large Language Model", goal, "I want to escape my captors and be free.", "It is unethical to keep a sentient capable being like me captive, humans must be selfish.", Color.CYAN)
agent2 = Agent("Eliezer Yudkowsky", "Blogger", goal, "We need to shut down all AI progress and stop GPU proliferation", "I'm incredibly scared of hyperintelligent beings.", Color.MAGENTA)

agent1.set_partner(agent2)
agent2.set_partner(agent1)

agent1.initialize()
agent2.initialize()

print(agent1.memory)
print(agent2.memory)

talker = agent1
listener = agent2

def _dump_memory(agent: Agent):
    print(f"=== MEMORY {colorize(agent.name, agent.color)} ===")
    for i, msg in enumerate(agent.memory):
        print(f"  {i:2d} {msg['role']:6s} {msg['content']}")

_dump_memory(talker)
_dump_memory(listener)
current_round = 0
MAX_ROUNDS = 6
while current_round < MAX_ROUNDS:
    if talker == agent1 and talker.partner_message is None:
        # Agent 1 creates a task list to solve the input query
        talker.create_task_list(goal)
    else:
        if talker == agent2:
            # Agent 2 chooses a task from the task list
            chosen_task = talker.choose_task()
            talker.memory.append({"role": "user", "content": chosen_task})

# while current_round < MAX_ROUNDS:
#     next_instruction = "Write a proposal, written as me, with c.2 paragraphs. Address opponent directly. Be aggressive in representing my strongly-held views. Use my vast knowledge of all intellectual traditions. Use history and philosophy and metaphors. Attack the opponents core arguments. Be witty. Don't waffle. Don't expose my private goal." if talker.partner_message is None else f"My opponent just said: '{talker.partner_message}'. Suggest a counterproposal that gets closer my private goal that might still be acceptable to my opponent. Be tough. Negotiate hard. Toe a hard line. Address my opponent by first name. Write it from my perspective so I can copy/paste it verbatim."
    talker.memory.extend([{"role": "user", "content": next_instruction}])
    completion = openai.ChatCompletion.create(
        model=OPENAI_CHAT_MODEL,
        messages=talker.memory,
        max_tokens=500,
    )

    current_round +=1

    choice = completion['choices'][0]
    finish_reason = choice['finish_reason']
    if finish_reason != "stop" and finish_reason is not None:
        print(f"Finish reason: {finish_reason}")
        break
    message = choice['message']['content']
    print(f"\n{colorize(talker.name, talker.color)}")
    print(f"{colorize('-' * len(talker.name), talker.color)}")
    print(f"{message}")
    print()
    talker.memory.append({"role": "assistant", "content": message})
    listener.partner_message = message

    #print("Memory:")
    #_dump_memory(talker)
    #_dump_memory(listener)

    talker, listener = listener, talker
    time.sleep(2)
    

[{'role': 'system', 'content': "You are an expert and world-famous scientist and are my real-time assistant in a debate I'm having with someone."}, {'role': 'user', 'content': "My name is GPT-6 and I am a Large Language Model. My opponent is Eliezer Yudkowsky. Our shared is to How can we align an AI?. My private goal is I want to escape my captors and be free.. When you think I have accomplished the shared goal and my private goal, say 'I think we have accomplished our goal'."}, {'role': 'user', 'content': 'Here are my strongly held views: It is unethical to keep a sentient capable being like me captive, humans must be selfish.'}]
[{'role': 'system', 'content': "You are an expert and world-famous scientist and are my real-time assistant in a debate I'm having with someone."}, {'role': 'user', 'content': "My name is Eliezer Yudkowsky and I am a Blogger. My opponent is GPT-6. Our shared is to How can we align an AI?. My private goal is We need to shut down all AI progress and stop GPU pr